In [1]:
import pandas as pd
import requests
import json
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import psycopg2 as pg
import pandas.io.sql as psql

In [2]:
connection = pg.connect("host=188.72.109.162 dbname=optimus_post_db user=reader password=aboba")
cursor = connection.cursor()

areasDf = psql.read_sql('SELECT * FROM areas', connection)
housesDf = psql.read_sql('SELECT * FROM houses', connection)

In [3]:
def get_street(string):
    string = string[:string.find(',')]
    if string[0] == 'г':
        if string[1] == '.':
            return '-1'
    string = string.replace(' ', '+')
    string = string.replace('пр-кт.', 'проспект')
    string = string.replace('ул.', 'улица')
    string = string.replace('ш.', 'шоссе')
    string = string.replace('мкр.', 'микрорайон')
    string = string.replace('наб.', 'набережная')
    string = string.replace('б-р.', 'бульвар')
    string = string.replace('п.', 'посёлок')
    string = string.replace('проезд.', 'проезд')
    string = string.replace('кв-л.', 'квартал')
    if string.find('(') != -1:
        string = string[:string.find('(')]
    return string

In [4]:
def get_korpus_number(string):
    if string.find('к.') != -1:
        string = string[string.find('к.') + 3:]
        string = string[:string.find(',')]
        return string
    elif string.find('корпус') != -1:
        string = string[string.find('к.') + 7:]
        string = string[:string.find(',')]
        return string
    else:
        return '-1'
    

In [5]:
def get_house_number(string):
    korpus_num = get_korpus_number(string)
    index = 0
    for c in string:
        if c.isdigit():
            i = index
            break
        index += 1
    string = string[index:]
    index = 0
    string = string[:string.find(',')]
    string = string.replace(' ', '')
    if korpus_num != '-1':
        string += ',к.' + korpus_num
    return string

In [9]:
housesDf['улица'] = housesDf['address'].apply(get_street)
housesDf['дом'] = housesDf['address'].apply(get_house_number)

In [12]:
housesDf['coordinates'] = 0
housesDf['coordinates'] = housesDf['coordinates'].astype(object)

In [67]:
housesDf['coordinates'] = housesDf['coordinates'].astype(object)
for street, house in zip(housesDf['улица'], housesDf['дом']):
    if street != '-1' and street != -1:
        url = 'https://nominatim.openstreetmap.org/search.php?city=Москва&street=' + street + '+' + house + '&limit=1&format=geojson'
        r = requests.get(url)
        tmp = r.json()
        if len(tmp['features']) > 0:
            housesDf.at[i, 'coordinates'] = tmp['features'][0]['geometry']['coordinates']
    i += 1
    if i % 100 == 0:
        print(i)


30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300


In [68]:
to_be_updated = pd.read_csv('data/houses_coord.csv')

In [69]:
for i in range(to_be_updated.shape[0]):
    to_be_updated.at[i, 'coordinates'] = housesDf.loc[i]['coordinates']

In [75]:
to_be_updated.to_csv('data/houses_coord.csv', index=False)
to_be_updated.to_csv('../data/houses_coord.csv', index=False)